In [27]:
from utils.tools import dotdict

args = dotdict()
args.target = 'close'
args.des = 'test'
args.dropout = 0.05
args.num_workers = 10
args.gpu = 0
args.lradj = 'type1'
args.devices = '0'
args.use_gpu = True
args.use_multi_gpu = False
args.freq = 't'
args.checkpoints = './checkpoints/'
args.bucket_size = 4
args.n_hashes = 4
args.is_trainging = True
args.root_path = './data/'
args.data_path ='ACBI_test.csv'
args.model_id='Financial_96_24'
args.model = 'Autoformer'
args.data = 'custom'
args.features = 'M'
args.seq_len = 96
args.label_len = 48
args.pred_len = 24
args.e_layers = 2
args.d_layers = 1
args.n_heads = 8
args.factor = 1
args.enc_in = 5
args.dec_in =5
args.c_out = 5
args.d_model = 512
args.des = 'Exp'
args.itr = 1
args.d_ff = 2048
args.moving_avg = 25
args.factor = 1
args.distil = True
args.output_attention = False
args.patience= 3
args.learning_rate = 0.0001
args.batch_size = 32
args.embed = 'timeF'
args.activation = 'gelu'
args.use_amp = False
args.loss = 'mse'
args.train_epochs = 10

In [28]:
setting = '{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
    args.model_id,
    args.model,
    args.data,
    args.features,
    args.seq_len,
    args.label_len,
    args.pred_len,
    args.d_model,
    args.n_heads,
    args.e_layers,
    args.d_layers,
    args.d_ff,
    args.factor,
    args.embed,
    args.distil,
    args.des, 1)
setting

'Financial_96_24_Autoformer_custom_ftM_sl96_ll48_pl24_dm512_nh8_el2_dl1_df2048_fc1_ebtimeF_dtTrue_Exp_1'

In [29]:
from models.Autoformer import Model as Autoformer
import torch


model = Autoformer(args)
model.load_state_dict(torch.load('checkpoints/{}/checkpoint.pth'.format(setting)))

<All keys matched successfully>

In [30]:
from data_provider.data_loader import Dataset_Custom

data_set = Dataset_Custom(
    root_path=args.root_path,
    data_path=args.data_path,
    flag="train",
    size=[args.seq_len, args.label_len, args.pred_len],
    features=args.features,
    target=args.target,
    timeenc= 0 if args.embed != 'timeF' else 1,
    freq=args.freq
)
len(data_set)

2013

In [31]:
seq_x, seq_y, seq_x_mark, seq_y_mark = data_set[0]
seq_x.shape, seq_y.shape, seq_x_mark.shape, seq_y_mark.shape

((96, 5), (72, 5), (96, 5), (72, 5))

In [32]:
from torch.utils.data import DataLoader

data_loader = DataLoader(data_set, batch_size=1, shuffle=True)
len(data_loader)

2013

In [40]:
batch_x , batch_y, batch_x_mark, batch_y_mark = next(iter(data_loader))
batch_x.shape, batch_y.shape, batch_x_mark.shape, batch_y_mark.shape

(torch.Size([1, 96, 5]),
 torch.Size([1, 72, 5]),
 torch.Size([1, 96, 5]),
 torch.Size([1, 72, 5]))

In [41]:
batch_x = batch_x.float()
batch_y = batch_y.float()
batch_x_mark = batch_x_mark.float()
batch_y_mark = batch_y_mark.float()

In [35]:
import os
import matplotlib.pyplot as plt

def visualize_pred(batch_x, batch_y, output, result_dir="img"):
    os.makedirs(result_dir, exist_ok=True)
    batch_x_np = batch_x.cpu().detach().numpy()
    batch_y_np = batch_y.cpu().detach().numpy()
    output_np = output.cpu().detach().numpy()

    seq_len = len(batch_x_np[0, :, 0])      # longueur de la séquence d'entrée
    out_len = len(output_np[0, :, 0])       # longueur de la séquence de sortie

    for i in range(batch_x_np.shape[-1]):
        plt.figure(figsize=(20, 10))

        # Affichage de la séquence d'entrée
        plt.plot(range(seq_len), batch_x_np[0, :, i], label='Input Sequence')

        # Affichage de la séquence de sortie réelle en continuité de la séquence d'entrée
        plt.plot(range(seq_len - 1, seq_len + out_len - 1), batch_y_np[0, -out_len:, i], label='True Output Sequence')

        # Affichage de la séquence de sortie prédite en continuité de la séquence d'entrée
        plt.plot(range(seq_len - 1, seq_len + out_len - 1), output_np[0, :, i], label='Predicted Output Sequence')

        # Mise en forme de la figure
        plt.xlabel('Time Steps')
        plt.ylabel('Values')
        plt.legend()
        plt.savefig(f'{result_dir}/pred_{i}.png')
        plt.close()

In [42]:
output = model(batch_x, batch_x_mark, batch_y, batch_y_mark)
output.shape

torch.Size([1, 24, 5])

In [43]:
visualize_pred(batch_x, batch_y, output,result_dir="financial")